In [4]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Function to clean NaN and infinite values
def clean_data(X_train, X_valid, X_test):
    X_train.fillna(X_train.mean(), inplace=True)
    X_valid.fillna(X_valid.mean(), inplace=True)
    X_test.fillna(X_test.mean(), inplace=True)
    return X_train, X_valid, X_test


# Load data
data_train = pd.read_pickle("clean_data/train/all_data_train.pkl")
data_valid = pd.read_pickle("clean_data/valid/all_data_valid.pkl")
data_test = pd.read_pickle("clean_data/test/all_data_test.pkl")

# Split the data into features (X) and target (y)
X_train = data_train.drop("class", axis=1)
X_valid = data_valid.drop("class", axis=1)
X_test = data_test.drop("class", axis=1)

y_train = data_train["class"]
y_valid = data_valid["class"]
y_test = data_test["class"]

# Encode the target variable (multi-class encoding)
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoded_y_valid = encoder.transform(y_valid)
encoded_y_test = encoder.transform(y_test)

# Clean the data (replace NaNs and infinities)
X_train, X_valid, X_test = clean_data(X_train, X_valid, X_test)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Reshape the data to 3D for GRU (add a time step dimension)
X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_valid_reshaped = X_valid.reshape(X_valid.shape[0], 1, X_valid.shape[1])
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [6]:
# Function to build a GRU model with variable layers and neurons
def build_gru_model(
    num_neurons=32, num_layers=1, dropout_rate=0.2, input_shape=(None,)
):
    model = Sequential()
    for i in range(num_layers):
        if i == 0:
            model.add(
                GRU(
                    num_neurons,
                    return_sequences=(num_layers > 1),
                    input_shape=input_shape,
                )
            )
        else:
            model.add(GRU(num_neurons, return_sequences=(i != num_layers - 1)))
        model.add(Dropout(dropout_rate))

    # Output layer for classification
    model.add(Dense(4, activation="softmax"))

    # Compile the model
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return model

In [7]:
# Function to perform manual grid search
def manual_grid_search(X_train, y_train, X_valid, y_valid, param_grid):
    best_accuracy = 0
    best_model = None
    best_params = {}

    for num_neurons in param_grid["num_neurons"]:
        for num_layers in param_grid["num_layers"]:
            for dropout_rate in param_grid["dropout_rate"]:
                for batch_size in param_grid["batch_size"]:
                    print(
                        f"Training model with {num_neurons} neurons, {num_layers} layers, {dropout_rate} dropout, and {batch_size} batch size"
                    )

                    # Build and train the model
                    input_shape = (
                        1,
                        X_train.shape[2],
                    )  # Adjust input shape to match your data
                    model = build_gru_model(
                        num_neurons, num_layers, dropout_rate, input_shape=input_shape
                    )
                    model.fit(
                        X_train,
                        y_train,
                        epochs=30,
                        batch_size=batch_size,
                        validation_data=(X_valid, y_valid),
                        verbose=1,
                    )

                    # Evaluate the model on validation data
                    _, accuracy = model.evaluate(X_valid, y_valid)
                    print(f"Validation accuracy: {accuracy}")

                    # Keep track of the best model
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_model = model
                        best_params = {
                            "num_neurons": num_neurons,
                            "num_layers": num_layers,
                            "dropout_rate": dropout_rate,
                            "batch_size": batch_size,
                        }

    return best_model, best_accuracy, best_params

In [8]:
# Define the hyperparameter grid manually
param_grid = {
    "num_neurons": [32, 64, 128],
    "num_layers": [1, 2],
    "dropout_rate": [0.1, 0.2, 0.3],
    "batch_size": [32, 64],
}

# Perform the manual grid search
best_model, best_accuracy, best_params = manual_grid_search(
    X_train_reshaped, encoded_y_train, X_valid_reshaped, encoded_y_valid, param_grid
)

# Display the best result
print(f"Best validation accuracy: {best_accuracy}")
print(f"Best parameters: {best_params}")

# Evaluate the best model on the test data
_, test_accuracy = best_model.evaluate(X_test_reshaped, encoded_y_test)
print(f"Test accuracy: {test_accuracy}")

# Predict on the test data
y_pred = np.argmax(best_model.predict(X_test_reshaped), axis=1)

# Generate confusion matrix
cm = confusion_matrix(encoded_y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="g")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

Training model with 32 neurons, 1 layers, 0.1 dropout, and 32 batch size
Epoch 1/30
375/375 [==============================] - 7s 8ms/step - loss: 1.1325 - accuracy: 0.5622 - val_loss: 0.8485 - val_accuracy: 0.7401
Epoch 2/30
375/375 [==============================] - 2s 5ms/step - loss: 0.7458 - accuracy: 0.7587 - val_loss: 0.5394 - val_accuracy: 0.8415
Epoch 3/30
375/375 [==============================] - 2s 6ms/step - loss: 0.5455 - accuracy: 0.8234 - val_loss: 0.4063 - val_accuracy: 0.8743
Epoch 4/30
375/375 [==============================] - 2s 5ms/step - loss: 0.4584 - accuracy: 0.8439 - val_loss: 0.3466 - val_accuracy: 0.8813
Epoch 5/30
375/375 [==============================] - 2s 5ms/step - loss: 0.4134 - accuracy: 0.8547 - val_loss: 0.3181 - val_accuracy: 0.8843
Epoch 6/30
375/375 [==============================] - 2s 6ms/step - loss: 0.3836 - accuracy: 0.8630 - val_loss: 0.2946 - val_accuracy: 0.8896
Epoch 7/30
375/375 [==============================] - 2s 5ms/step - loss: 0

KeyboardInterrupt: 